In [1]:
import pandas as pd

# 1 read construction material index index dataset

In [2]:
df_construction_material_index= pd.read_excel(r'C:\Users\fnafisa\Construction Material Price Index\data\training data\target\construction material price index.xlsx')

clean the dataset

In [3]:
# Remove leading and trailing spaces from all string columns
df_construction_material_index = df_construction_material_index.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)

# Convert all characters in 'Material_EN' column to lowercase
df_construction_material_index["Material_EN"] = df_construction_material_index["Material_EN"].str.lower()

# Remove time from Date column in df_construction_material_index
df_construction_material_index['Date'] = pd.to_datetime(df_construction_material_index['Date']).dt.strftime('%Y-%m-%d')

prepare material datsets

In [4]:
set_material_names= set(df_construction_material_index["Material_EN"])

ls_material_lists= {}
for material in set_material_names:
    ls_material_lists[material] = df_construction_material_index[df_construction_material_index["Material_EN"]== material]

# 2 prepare the features

## features from investing.com: Bren Oil, Gold, and TASI

Brent oil

In [5]:
df_feature_brent_oil= pd.read_csv(r'C:\Users\fnafisa\Construction Material Price Index\data\training data\features\Brent Oil Futures Historical Data.csv')

In [6]:
# Convert Date column to YYYY-MM-DD format
df_feature_brent_oil['Date'] = pd.to_datetime(df_feature_brent_oil['Date']).dt.strftime('%Y-%m-%d')
df_feature_brent_oil= df_feature_brent_oil[['Date', 'Price']]
df_feature_brent_oil['Price']= df_feature_brent_oil['Price'] * 3.75  # Convert from USD to SAR assuming 1 USD = 3.75 SAR

gold

In [7]:
df_gold= pd.read_csv(r'C:\Users\fnafisa\Construction Material Price Index\data\training data\features\Gold Futures Historical Data.csv')

In [8]:
# Convert Date column to YYYY-MM-DD format
df_gold['Date'] = pd.to_datetime(df_gold['Date']).dt.strftime('%Y-%m-%d')
df_gold = df_gold[['Date', 'Price']]
df_gold = df_gold.apply(lambda x: x.str.replace(",", "") if x.dtype == 'object' else x)
# Convert from USD to SAR assuming 1 USD = 3.75 SAR
df_gold['Price'] = df_gold['Price'].astype(float) # Convert to float
df_gold['Price'] = df_gold['Price'] * 3.75  # Convert to SAR

TASI

In [9]:
df_TASI= pd.read_csv(r"C:\Users\fnafisa\Construction Material Price Index\data\training data\features\Tadawul All Share Historical Data.csv")

In [10]:
# Convert Date column to YYYY-MM-DD format
df_TASI['Date'] = pd.to_datetime(df_TASI['Date']).dt.strftime('%Y-%m-%d')
df_TASI = df_TASI[['Date', 'Price']]
df_TASI = df_TASI.apply(lambda x: x.str.replace(",", "") if x.dtype == 'object' else x)

Fed Interests rate

In [11]:
df_fedfund_rates= pd.read_excel(r"C:\Users\fnafisa\Construction Material Price Index\data\training data\features\FEDFUNDS.xlsx")

In [12]:
df_fedfund_rates.columns= ['Date', 'Price']
df_fedfund_rates['Date'] = pd.to_datetime(df_fedfund_rates['Date']).dt.strftime('%Y-%m-%d')

Merge all the features

In [13]:
# Rename Price columns to distinguish them
df_feature_brent_oil = df_feature_brent_oil.rename(columns={'Price': 'brent_oil_price'})
df_gold = df_gold.rename(columns={'Price': 'gold_price'})
df_TASI = df_TASI.rename(columns={'Price': 'tasi_price'})
df_fedfund_rates = df_fedfund_rates.rename(columns={'Price': 'fed_rate'})

# Merge the dataframes on 'Date' using inner join to get only common dates
df_merged = df_feature_brent_oil.merge(df_gold, on='Date', how='inner')
df_merged = df_merged.merge(df_TASI, on='Date', how='inner')
df_merged = df_merged.merge(df_fedfund_rates, on='Date', how='inner')

# Sort by Date
df_merged = df_merged.sort_values('Date').reset_index(drop=True)



In [14]:
# Inner merge each material dataframe with df_merged on 'Date'
for material in ls_material_lists:
    ls_material_lists[material] = ls_material_lists[material].merge(df_merged, on='Date', how='inner')

ls_material_lists

{'120mm national electrical cables':     Category_En Category                Material_AR  \
 0        Cables    كيابل  كيابل كهربائية 120مم وطني   
 1        Cables    كيابل  كيابل كهربائية 120مم وطني   
 2        Cables    كيابل  كيابل كهربائية 120مم وطني   
 3        Cables    كيابل  كيابل كهربائية 120مم وطني   
 4        Cables    كيابل  كيابل كهربائية 120مم وطني   
 ..          ...      ...                        ...   
 150      Cables    كيابل  كيابل كهربائية 120مم وطني   
 151      Cables    كيابل  كيابل كهربائية 120مم وطني   
 152      Cables    كيابل  كيابل كهربائية 120مم وطني   
 153      Cables    كيابل  كيابل كهربائية 120مم وطني   
 154      Cables    كيابل  كيابل كهربائية 120مم وطني   
 
                           Material_EN Unit_ar Unit        Date   Price  \
 0    120mm national electrical cables   1 متر  1 M  2023-01-01  196.61   
 1    120mm national electrical cables   1 متر  1 M  2022-01-01  195.72   
 2    120mm national electrical cables   1 متر  1 M  2023-02-01  

# 3 Save the lists in excel xlsx file

In [15]:
count= 0
for material in ls_material_lists:
    # we should join this with brent oil prices later
    ls_material_lists[material].to_excel(r'C:\Users\fnafisa\Construction Material Price Index\data\training data\combined\{}.xlsx'.format(material), index= False)
    count += 1
print(count)

37
